In [29]:
# for python 3.6
import os
from MisrToolkit import MtkFile, orbit_to_path, latlon_to_bls
import netCDF4
import numpy
import ssl
import shutil

ssl._create_default_https_context = ssl._create_unverified_context

WORK_SPACE = os.getcwd()
MISR_NC_FOLDER = '/disk1/Data/MISR4AHI2015070120210630_3'

AHI_RESOLUTION = 0.01

CHECK_SIZE = 0.04


def find_nearest_index(array, value):
    array = numpy.asarray(array)
    idx = (numpy.abs(array - value)).argmin()
    return idx


def get_roi_latlon_list(center_lat, center_lon):
    roi_ullat = center_lat + CHECK_SIZE/2
    roi_ullon = center_lon - CHECK_SIZE/2
    roi_lrlat = center_lat - CHECK_SIZE/2
    roi_lrlon = center_lon + CHECK_SIZE/2
    ahi_lats = numpy.arange(60. - AHI_RESOLUTION / 2, -60, -AHI_RESOLUTION)
    ahi_lons = numpy.arange(85. + AHI_RESOLUTION / 2, 205, AHI_RESOLUTION)
    n_lats = ahi_lats[find_nearest_index(ahi_lats, roi_ullat):find_nearest_index(ahi_lats, roi_lrlat) + 1]
    n_lons = ahi_lons[find_nearest_index(ahi_lons, roi_ullon):find_nearest_index(ahi_lons, roi_lrlon) + 1]
    return n_lats, n_lons


def is_hot_spot(cood_point, misr_orbit, misr_camera_index, misr_nc_filename):
    roi_c_lat = cood_point[1]
    roi_c_lon = cood_point[0]
    roi_lats, roi_lons = get_roi_latlon_list(roi_c_lat, roi_c_lon)
    # print(roi_ahi_sr)
    misr_path = orbit_to_path(misr_orbit)
    
    m_file2 = MtkFile(misr_nc_filename)
    m_grid11 = m_file2.grid('4.4_KM_PRODUCTS')
    misr_resolutionv3 = m_grid11.resolution
    m_field11 = m_grid11.field('GEOMETRY/Scattering_Angle[' + str(misr_camera_index) + ']')
    
    # MISR data at ROI
    roi_misr_brfv3 = numpy.zeros((len(roi_lats), len(roi_lons)))
    for lat_index in range(len(roi_lats)):
        for lon_index in range(len(roi_lons)):
            lat = roi_lats[lat_index]
            lon = roi_lons[lon_index]
            
            try:
                misr_blsv3 = latlon_to_bls(misr_path, misr_resolutionv3, lat, lon)
                block_llv3 = misr_blsv3[0]
                b_lat_idxv3 = round(misr_blsv3[1])
                b_lon_idxv3 = round(misr_blsv3[2])

                block_brf_dnv3 = m_field11.read(block_llv3, block_llv3)[0]
                roi_brf_dnv3 = block_brf_dnv3[b_lat_idxv3][b_lon_idxv3]
                roi_misr_brfv3[lat_index][lon_index] = roi_brf_dnv3*0.1
            except Exception as e:
                roi_misr_brfv3[lat_index][lon_index] = 0.

    # if any cloud-free obs. is existed
    roi_misr_brfv3 = roi_misr_brfv3.flatten()
    if roi_misr_brfv3.max() > 175.:
        print(roi_misr_brfv3.max())
        return 1
    return 0


def is_valuable_record(cood_point, misr_path_orbit_camera):
    misr_path_orbit = misr_path_orbit_camera[:12]
    misr_orbit = int(misr_path_orbit[-6:])
    misr_camera_index = int(misr_path_orbit_camera[-1:])
    misr_nc_filename = os.path.join(MISR_NC_FOLDER, 'MISR_AM1_AS_LAND_' + misr_path_orbit + '_F08_0023.nc')
    to_record = 0
    band_indes = [2, 3]
    has_value = is_hot_spot(cood_point, misr_orbit, misr_camera_index, misr_nc_filename)
    if has_value > 0:
        to_record = has_value
    return to_record

In [32]:
if __name__ == "__main__":

#     folder_l1_list = ['0', '26', '45', '60', '70']
#     folder_l2_list = ['0', '1']
    folder_l1_list = ['45']
    folder_l2_list = ['0', '1']

    for folder_l1 in folder_l1_list:
        folder_l1_path = os.path.join(WORK_SPACE, folder_l1)
        for folder_l2 in folder_l2_list:
            folder_l2_path = os.path.join(folder_l1_path, folder_l2)
            roi_folder_list = os.listdir(folder_l2_path)
            for roi_folder in roi_folder_list:
                roi_folder_path = os.path.join(folder_l2_path, roi_folder)
                roi_infos = roi_folder.split('_')
                roi_name = roi_folder
                misr_vza_str = roi_infos[0]
                cood_point = [float(roi_infos[3]), float(roi_infos[2])] # lon lat
                misr_ray_matched_npy_filename = os.path.join(roi_folder_path, roi_name + '_matched_record.npy')
                
                matched_record = numpy.load(misr_ray_matched_npy_filename, allow_pickle=True)
                roi_valuable_record = []
                record_str = ''
                if len(matched_record) > 0:
                    matched_roi_info = matched_record[0]
                    roi_misr_infos = matched_roi_info['roi_misr_infos']
                    for roi_misr_info in roi_misr_infos:
                        misr_path_orbit_camera = roi_misr_info['misr_path_orbit_camera']
                        matched_info = roi_misr_info['matched_info']
                        is_hot_sopt = is_valuable_record(cood_point, misr_path_orbit_camera)
                        if is_hot_sopt:
                            print(roi_folder)
                            print(matched_info)
                            camera_idx = matched_info[2]
                            ahi_obs_time = matched_info[4]
                            show_b3_file = os.path.join(roi_folder_path, ahi_obs_time + '_band3_' + camera_idx + '.npy')
                            show_b4_file = os.path.join(roi_folder_path, ahi_obs_time + '_band4_' + camera_idx + '.npy')
                            if os.path.exists(show_b3_file):
                                print('rm:', show_b3_file)
                                os.remove(show_b3_file)
                            if os.path.exists(show_b4_file):
                                print('rm:', show_b4_file)
                                os.remove(show_b4_file)
                            


175.5
45_10_-34.65_118.35
['109', '104904', '2', '201909080201', '201909080200', '48.169', '46.748', '34.819', '35.785']
175.4
45_8_-34.75_118.35
['109', '104904', '2', '201909080201', '201909080200', '47.981', '46.839', '33.925', '35.717']
175.10000000000002
45_8_-34.85_118.35
['109', '104904', '2', '201909080201', '201909080200', '47.963', '46.947', '33.850', '35.635']
175.4
45_10_-34.45_116.75
['110', '94288', '2', '201709090207', '201709090210', '48.144', '47.470', '34.706', '38.041']
175.10000000000002
45_10_-34.45_116.75
['110', '102676', '2', '201904080207', '201904080210', '48.275', '47.470', '35.331', '38.041']
175.10000000000002
45_10_-34.45_116.75
['110', '104773', '2', '201908300207', '201908300210', '48.350', '47.470', '35.662', '38.041']
175.4
45_8_-34.35_116.75
['110', '94288', '2', '201709090207', '201709090210', '48.181', '47.365', '34.869', '38.127']
175.20000000000002
45_8_-34.35_116.75
['110', '102676', '2', '201904080207', '201904080210', '48.325', '47.365', '35.50